In [84]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import spacy
import csv

In [85]:
url = "https://dk.trustpilot.com/review/www.kmd.dk"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

review_divs = soup.find_all("div", {"class": "styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ"})

reviews = []

for div in review_divs:
    review_title = div.find("h2", {"class": "review-title"})
    
    # Find the first p tag within the div
    review_paragraph = div.find("p")
    reviews.append(review_paragraph.text)
    
print(reviews[1])

Jeg er desværre daglig bruger af KMD grundet mit arbejde. Det er helt vildt så dårlig KMD systemer fungerer. Der er ret mange basale ting der ikke fungerer, så som at man ikke kan kopier cpr nr. ind fra regneark ind i søgefeltet, og hver gang der kommer en opdatering bortfalder der vigtige funktioner. Det er som om de bare prøver at gøre det dårligere og dårligere. Ganske enkelt talentløst!


## Preprocessing

In [86]:
# Load danish tokenizer
nlp = spacy.load("da_core_news_sm")

In [87]:
def preprocess_text(text):
    
    # Tokenize the text and remove stop words and punctuation
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]

    return tokens 

In [88]:
print(preprocess_text(reviews[0]))

['600', 'menneske', 'middelfart', 'kommune', 'få', 'udbetale', 'ydelse', 'går', 'grunde', 'teknisk', 'fejl', 'kmd', 'side', 'betyde', 'modtage', 'rykkere', 'eventuelt', 'risikere', 'diverse', 'ret', 'ubehagelig', 'oplevelse', 'grund', 'få', 'ydelse', 'betale', 'regninger', 'dag', '1', 'november', 'ydelse', 'fortsat', 'komme', 'fuldstændigt', 'uacceptabel', 'kmd', 'tænke', 'hæfte', 'rykkere', 'nej', 'vel']


### Sentiment Analysis

In [89]:
# Load the DSL lexicon
dsl_lexicon = {}
with open('data/2_headword_headword_polarity.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        lemma, polarity = row[0], float(row[4])
        dsl_lexicon[lemma] = polarity

# Define a function to classify a review as positive, negative or neutral
def classify_review(review):
    tokens = preprocess_text(review)
        
    # Calculate the sentiment score of the review
    sentiment_score = sum([float(dsl_lexicon.get(token, 0)) for token in tokens])
    print(sentiment_score)
    
    # Classify the review based on the sentiment score
    if sentiment_score > 0:
        return "positive"
    elif sentiment_score < 0:
        return "negative"
    else:
        return "neutral"

# Test the function with a list of sample reviews
for review in reviews:
    classification = classify_review(review)
    print(classification)
    print(f'{review} \n ')
    


-6.0
negative
Over 600 mennesker i Middelfart kommune har ikke fået udbetalt deres ydelse i går grundet en teknisk fejl fra KMDs side.Dette betyder de vil modtage rykkere, og eventuelt risikere diverse ret så ubehagelige oplevelser på grund af de ikke har fået deres ydelse og hermed ikke kan betale deres regninger.Det er i dag den 1 November og deres ydelse er fortsat ikke kommet. Fuldstændigt uacceptabelt.Har KMD tænkt sig at hæfte for rykkere? Nej vel. 
 
-1.0
negative
Jeg er desværre daglig bruger af KMD grundet mit arbejde. Det er helt vildt så dårlig KMD systemer fungerer. Der er ret mange basale ting der ikke fungerer, så som at man ikke kan kopier cpr nr. ind fra regneark ind i søgefeltet, og hver gang der kommer en opdatering bortfalder der vigtige funktioner. Det er som om de bare prøver at gøre det dårligere og dårligere. Ganske enkelt talentløst! 
 
-4.0
negative
Urimelig behandling - KMD står for at lave hjemmeside til nemrefusion - hjemmeside virker ikke igen. Skriver fler

In [36]:
def check_word_polarity():
    # Load the DSL lexicon from the CSV file
    dsl_lexicon = {}
    with open('data/2_headword_headword_polarity.csv', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            lemma, polarity = row[0], float(row[4])
            dsl_lexicon[lemma] = polarity

    # Test the lexicon by printing the polarity score of a few words
    print(dsl_lexicon.get("løn", 0))
    print(dsl_lexicon.get("stjerne", 0))
    
check_word_polarity()

2.0
3.0
